**6/29/21**

I'm interested in doing a more a more robust analysis of how including all the strains of a species in a database affects search performance. I'm going to build tailored databases for each sample, using only a single genome for each organism. So it will be more useful for others, I'll use the reference genome for each species so I can make inferences about how databases using only reference genomes for the microbes in a sample would compare to using the proteins from all strains.

The purpose of this notebook is to document construction of these "Reference Genome" databases that also include human/contaminant sequences, and refining them through a two-step search. I'll then do the analysis and comparisons to the Public_Tailored databases in a separate notebook.

In [1]:
from elliot_utils import *
initWithReference()

In [2]:
analysisPath = Path.cwd().joinpath('analysis_files/reference_genome_db/')
sequenceData = Path('C:/Users/emlee/Documents/MSGFp/Sequences/Bacteria/AllNCBI_12-19/')
humanFile = Path.cwd().joinpath('../PublicSequences/Human9606_2-6-2019_TrypPigBov.fasta')

In [3]:
rawDBPath = Path.cwd().joinpath('../1-12-22_NextflowMSGF_Reference_Combined/databases/')
refinedDBPath = Path.cwd().joinpath('../1-12-22_NextflowMSGF_Reference_Combined/databases_refined/')

In [4]:
# Get species -> reference genome file associations
referenceGenomes = {} # key=species name, value=reference strain
with analysisPath.joinpath('reference_genomes.csv').open(mode='r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        referenceGenomes[row[0]] = row[1]

In [5]:
names = set()
for p in sequenceData.rglob('*'):
    if p.suffix == '':
        nameArray = p.name.split('_')
        names.add(nameArray[0] + ' ' + nameArray[1])

In [6]:
referenceFiles = {} # key=species name, value=file path of reference genome .fasta
for p in sequenceData.rglob('*'):
    if not p.suffix == '.fasta':
        continue
    with open(p, 'r') as database:
        rawText = database.read()
        dataList = rawText.split('\n>')
        dataList[0] = dataList[0][1:]
        del rawText
        newProt = Protein(dataList[0])
        taxa = newProt.taxa[0]
        taxaArray = taxa.split(' ')
        taxaName = taxaArray[0] + ' ' + taxaArray[1]
        if taxaName in referenceGenomes.keys():
            refStrain = referenceGenomes[taxaName]
            if refStrain == 'A' or taxa.find(refStrain) != -1:
                referenceFiles[taxaName] = p
referenceFiles['Eubacterium sp'] = sequenceData.joinpath('Eubacterium_sp/EubacteriumSp._SAMN10239549.fasta')
referenceFiles['Porphyromonas sp'] = sequenceData.joinpath('Porphyromonas_sp/PorphyromonasSp._SAMN03004273.fasta')
referenceFiles['Sutterella sp'] = sequenceData.joinpath('Sutterella_sp/SutterellaSp._SAMN11897248.fasta')

In [7]:
# Double check I have a reference genome file for each organism in the referenceGenomes dictionary
for taxa in referenceGenomes.keys():
    if not taxa in referenceFiles.keys():
        print(taxa)

In [8]:
# Get the species that go into each sample
sampleLists = {} # key=sampleID, value=set of species in the sample
with Path.cwd().joinpath('analysis_files/tailored_db_building/Tailored0_1.csv').open(mode='r', encoding='utf-8-sig') as infile:
    reader = csv.reader(infile)
    for row in reader:
        sampleID = row[0].split('_')[0]
        sampleLists[sampleID] = set()
        for i in range(1, len(row)):
            if row[i] == '':
                break
            elif row[i] == 'Null':
                continue
            sampleLists[sampleID].add(row[i])

In [9]:
# Get human and contaminant data from the file
humanData = ''
with humanFile.open(mode='r') as infile:
    humanData = infile.read()

In [55]:
# Write the initial databases
for s in SAMPLE_NAMES:
    bacterialProts = {} # key=protein sequence, value=protein object
    for species in sampleLists[s]:
        if not species in referenceFiles.keys():
            continue
        with open(referenceFiles[species], 'r') as database:
            rawText = database.read()
            dataList = rawText.split('\n>')
            dataList[0] = dataList[0][1:]
            del rawText
            for sequence in dataList:
                newProt = Protein(sequence)
                if newProt.sequence in bacterialProts.keys():
                    bacterialProts[newProt.sequence].addTaxa(newProt.taxa[0])
                else:
                    bacterialProts[newProt.sequence] = newProt
    toWrite = [humanData]
    for prot in bacterialProts.values():
        toWrite.append(prot.getEntry())
    with open(rawDBPath.joinpath(f'{s}_referenceGenome.fasta'), mode='w', newline='') as output:
        output.write(''.join(toWrite))

In [13]:
# Make the refined reference databases
unprocessed = getOrderedFiles(Path.cwd().joinpath('../1-12-22_NextflowMSGF_Reference_Combined/output/'), '.tsv')
processedPath = Path.cwd().joinpath('../1-12-22_NextflowMSGF_Reference_Combined/output_processed/')

In [17]:
collapseRepeatRows(unprocessed, processedPath)

In [18]:
processed = getOrderedFiles(processedPath, '.tsv')
referenceGenomeDBs = getOrderedFiles(rawDBPath, '.fasta')
refinedDBPath = Path.cwd().joinpath('../1-12-22_NextflowMSGF_Reference_Combined/databases_refined/')

In [20]:
# For each result/db pair, pull out all the hit proteins in the result file, identify the proteins in the DB, then write them out to a new DB
for i in range(len(processed)):
    protsToInclude = set()
    with processed[i].open(mode='r') as infile:
        reader = csv.reader(infile, delimiter='\t')
        for row in reader:
            protType = determineIDType(row)
            if protType == 'first':
                continue
            hitProts = getProteinHitList(row, 'all')
            for hit in hitProts:
                if hit.find('XXX_') == -1: # Ignore decoy hits
                    protsToInclude.add(hit)
    protObjs = {} # key=protID, value=protein object
    with open(referenceGenomeDBs[i], 'r') as database:
        rawText = database.read()
        dataList = rawText.split('\n>')
        dataList[0] = dataList[0][1:]
        del rawText
        for sequence in dataList:
            newProt = Protein(sequence)
            if newProt.id in protsToInclude:
                if newProt.id in protObjs.keys():
                    for taxa in newProt.taxa:
                        protObjs[newProt.id].addTaxa(taxa)
                else:
                    protObjs[newProt.id] = newProt
    toWrite = []
    for prot in protObjs.values():
        toWrite.append(prot.getEntry())
    with open(refinedDBPath.joinpath(f'{SAMPLE_NAMES[i]}_ReferenceGenome_refined.fasta'), 'w', newline='') as output:
        output.write(''.join(toWrite))

In [21]:
refinedOutput = getOrderedFiles(Path.cwd().joinpath('../1-12-22_NextflowMSGF_Reference_Combined/output_refined/'), '.tsv')
refinedOutputProcessedDir = Path.cwd().joinpath('../1-12-22_NextflowMSGF_Reference_Combined/output_refined_processed/')

In [24]:
collapseRepeatRows(refinedOutput, refinedOutputProcessedDir)